# Creating an order for multiple items.

Import the DataAPI and OrdersAPI wrapper.

In [1]:
from cast_planet import DataAPI, OrdersApi

In [2]:
data_api = DataAPI(api_key='your-api-key-here')
orders_api = OrdersApi(api_key='your-api-key-here')

## Searching for items.
For this example, we want to get an image for each month, over the UA Farm.
To do this, we will modify some the same filters used in the basic example.

In [3]:
from cast_planet.data.filters.field import GeometryFilter, DateRangeFilter
from cast_planet.data.filters.logical import AndFilter

Since the area of interest will not be altered, we can create a single filter.

In [4]:
point_of_interest = {
    "type": "Point",
    "coordinates": [-94.16987610350382, 36.09799882432394]
}
aoi_filter = GeometryFilter(field_name='geometry', config=point_of_interest)

Now, we can create a loop to iterate over each month, create a new date filter and perform the search.

In [5]:
full_search_results = []
for i in range(1,13):
    # Create the date filter
    start_time = f'2023-{i:02}-15T00:00'
    end_time = f'2023-{i:02}-15T23:59'
    date_filter = DateRangeFilter(field_name='acquired', config={'gte': start_time, 'lte': end_time})
    
    # Combine the date filter with a logic filter
    search_filter = AndFilter(config=[aoi_filter,date_filter])
    
    # Perform the search.
    response = data_api.quick_search(item_types=['PSScene'], search_filter=search_filter)
    
    # Aggregate the search results to a single list.
    full_search_results.extend(response.features)

# Display the final result count
len(full_search_results)
    

12

## Create an order to request multiple assets at once. 
We can create an order by sending a list of products that we want to acquire, along with the item type and [product bundle](https://developers.planet.com/apis/orders/product-bundles-reference/) needed.


To learn more about creating an order see the [Planet Orders API documentation](https://developers.planet.com/apis/orders/reference/#tag/Orders/operation/createOrder).
<br/>
To learn more about the CreateOrder data model, please see the package documentation. 


In [6]:
from cast_planet.orders.models.order_new import CreateOrder

order_information = {
    "name" :f"Farm Monthly - 2023",
    "source_type": 'scenes',
    "order_type": "partial",
    "products": [
        {
            "item_ids": [x.id for x in full_search_results],
            "item_type": "PSScene",
            "product_bundle": "analytic_udm2",
        }
    ],
    "notifications": {
        'email': True
    }
}
order = orders_api.create_order(CreateOrder(**order_information))

## View order status and download results.
Once your order is created, you can see it's updated status by using the get_order_details_by_id method, or you can view your order in the [My Orders](https://www.planet.com/account/#/orders) section of your planet account dashboard.

In [7]:
order = orders_api.get_order_details_by_id('5bfa8287-f605-439e-8c55-523a8450f739')

Once your order is completed, you can download all the via the api with the download_order_results method. This will download ALL files produced by the order. 
<br/><br/>
If you find that you only want to download some files, or you would like to preview the items beforehand. You may view the order by clicking on order in the My Orders section of the dashboard, or going to https://www.planet.com/account/#/orders/your-order-id-here

In [ ]:
orders_api.download_order_results('5bfa8287-f605-439e-8c55-523a8450f739', 'order-example')

order-example/5bfa8287-f605-439e-8c55-523a8450f739/PSScene/20231215_161357_05_2415_3B_udm2.tif: 100%|##########| 6.25M/6.25M [00:00<00:00, 24.1MB/s]
order-example/5bfa8287-f605-439e-8c55-523a8450f739/PSScene/20231215_161357_05_2415_3B_AnalyticMS_metadata.xml: 100%|##########| 10.3k/10.3k [00:00<00:00, 10.7MB/s]
order-example/5bfa8287-f605-439e-8c55-523a8450f739/PSScene/20231215_161357_05_2415_3B_AnalyticMS.tif: 100%|##########| 494M/494M [00:14<00:00, 35.1MB/s] 
order-example/5bfa8287-f605-439e-8c55-523a8450f739/PSScene/20231215_161357_05_2415_metadata.json: 100%|##########| 961/961 [00:00<00:00, 10.5MB/s]
order-example/5bfa8287-f605-439e-8c55-523a8450f739/PSScene/20231015_160905_18_24a9_3B_udm2.tif: 100%|##########| 5.77M/5.77M [00:00<00:00, 24.4MB/s]
order-example/5bfa8287-f605-439e-8c55-523a8450f739/PSScene/20231015_160905_18_24a9_3B_AnalyticMS_metadata.xml: 100%|##########| 10.3k/10.3k [00:00<00:00, 22.7MB/s]
order-example/5bfa8287-f605-439e-8c55-523a8450f739/PSScene/20231015_16090